In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

img_path = 'chest_xray/train/NORMAL/IM-0115-0001.jpeg'
img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)

left_roi, right_roi = extract_lungs_roi(img)

left_lbp = compute_lbp_img(left_roi)
right_lbp = compute_lbp_img(right_roi)

left_final = pad_and_resize(left_lbp)
right_final = pad_and_resize(right_lbp)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(left_final, cmap='gray')
plt.title("Left LBP Padded")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(right_final, cmap='gray')
plt.title("Right LBP Padded")
plt.axis('off')

plt.tight_layout()
plt.savefig("LBP_ROIs.pdf", bbox_inches='tight')
plt.close()

In [ ]:
def save_lbp_tensors(img_folder,save_folder):
    os.makedirs(save_folder, exist_ok=True)

    for label in ['NORMAL','PNEUMONIA']:
        class_dir = os.path.join(img_folder, label)
        save_class_dir = os.path.join(save_folder, label)
        os.makedirs(save_class_dir, exist_ok=True)

        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)

                left_roi, right_roi = extract_lungs_roi(img)
                left_lpb = compute_lbp_img(left_roi)
                right_lpb = compute_lbp_img(right_roi)
                left_final = pad_and_resize(left_lpb)
                right_final = pad_and_resize(right_lpb)

                lbp_tensor = torch.stack([torch.tensor(left_final), torch.tensor(right_final)])

                save_path = os.path.join(save_class_dir, img_name.replace('.jpg', '.pt').replace('.png', '.pt').replace('.jpeg', '.pt'))
                torch.save(lbp_tensor, save_path)
                print(f"Saved LBP tensor: {save_path}")  




train_folder = 'chest_xray/train/'
save_folder = 'chest_xray/chest_xray_lbp_tensors/train/'

save_lbp_tensors(train_folder,save_folder)


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from PIL import Image
import random
import numpy as np

class HardNegativeSiameseDataset(Dataset):
    def __init__(self, root_dir, transform, hard_negative_mining=False):
        
        self.dataset = datasets.ImageFolder(root_dir, transform=transform)
        self.samples = self.dataset.samples
        self.targets = self.dataset.targets
        self.transform = transform
        self.hard_negative_mining = hard_negative_mining

        self.class_indices = {}
        for idx, (_, label) in enumerate(self.samples):
            self.class_indices.setdefault(label, []).append(idx)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path1, label1 = self.samples[idx]
        img1 = Image.open(path1).convert('L')
        img1 = self.transform(img1)
        lbp1 = compute_lbp(img1)
        
        is_positive_pair = random.random() < 0.5
        
        if is_positive_pair:
            label = -1.0 
            idx2 = idx
            while idx2 == idx:
                idx2 = random.choice(self.class_indices[label1])
        else:
            label = 1.0
            neg_labels = [l for l in self.class_indices if l != label1]
            neg_label = random.choice(neg_labels)
            candidates = self.class_indices[neg_label]

            if self.hard_negative_mining:
                idx2 = self._select_hard_negative(idx, candidates)
            else:
                idx2 = random.choice(candidates)

        path2, _ = self.samples[idx2]
        img2 = Image.open(path2).convert('L')
        img2 = self.transform(img2)
        lbp2 = compute_lbp(img2)
        
        return (img1, lbp1), (img2, lbp2), torch.tensor(label, dtype=torch.float32)

    def _select_hard_negative(self, idx, candidates):
        """Optional: pick a 'hard' negative close to idx"""
        return random.choice(candidates)